In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
from multiprocessing import Pool
import os 
import pandas as pd
import tqdm
from bishop_peaks_onsets import bishop_peaks_and_onsets
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion2 /')
from pathlib import Path
import traceback
from src.parsing.MESAParser import MESAParser
from scipy.signal import butter, sosfiltfilt, resample
import scipy

In [3]:
# dl1 = MESAParser()
# patients = dl1.database_all_patient_IDs
# peaks_path = "/home/kkotzen/leas_bishop_peaks/peaks"
# onsets_path = "/home/kkotzen/leas_bishop_peaks/onsets"

# fs = 256
# filter_lowcut=0.4
# filter_highcut=8
# filter_order=8
# sos = butter(filter_order, [filter_lowcut / (fs / 2), filter_highcut / (fs / 2)], 'bandpass', output='sos')

# def extract(patient):
#     try:
#         dl = MESAParser()
#         ppg = dl.load_signal(patient, 'Pleth')

#         ppg_filt = sosfiltfilt(sos, ppg, axis=0)  # , padlen=3 * (max(len(b), len(a)) - 1))

#         ppg_peaks, ppg_onsets = bishop_peaks_and_onsets(ppg_filt, fs=256, min_time_between_ms=20,
#                                     pre_filter=True, filter_lowcut=0.4, filter_highcut=7.99, filter_order=8,
#                                     resample=True, resample_fs=32,
#                                     window=100, hop=90, correct_points=True)
#         np.save(f"{peaks_path}/{patient}.npy", ppg_peaks)
#         np.save(f"{onsets_path}/{patient}.npy", ppg_onsets)
#     except:
#         print(f"Failed to process: {patient}")
#         traceback.print_exc()

# with Pool(64) as p:
#         p.map(extract, patients)

In [4]:
# from tensorflow import keras
# import numpy as np
# from joblib import Parallel, delayed

# class PPGPeakOnsetLoader(keras.utils.Sequence):
#     """Helper to iterate over the data (as Numpy arrays)."""
    
#     def __init__(self, batch_size, duration, hop, ppg_path, peak_path, patients):
        
#         self.fs = 256
#         self.resample_fs = 32
#         self.F = self.fs / self.resample_fs
#         self.n_pool = 32
        
#         filter_lowcut=0.4
#         filter_highcut=8
#         filter_order=8
#         self.sos = butter(filter_order, [filter_lowcut / (self.fs / 2), filter_highcut / (self.fs / 2)], 'bandpass', output='sos')
        
#         self.ppg_path = ppg_path
#         self.peak_path = peak_path
#         self.batch_size = batch_size
#         self.window = duration
#         self.hop = hop
#         self.patients = patients
        
#         self.data = self._load_data()
#         print("Datal loaded...")
#         self.labels = self._load_labels()
#         print("Labels loaded...")

#     def __len__(self):
#         samples = 0
#         for patient in self.patients:
#             samples += int(self.data[patient].shape[0]/(self.hop*self.fs))
#         return samples // self.batch_size

#     def __getitem__(self, idx):
#         """Returns tuple (input, target) correspond to batch #idx."""
#         i = idx * self.batch_size
#         batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
#         batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
#         x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
#         for j, path in enumerate(batch_input_img_paths):
#             img = load_img(path, target_size=self.img_size)
#             x[j] = img
#         y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
#         for j, path in enumerate(batch_target_img_paths):
#             img = load_img(path, target_size=self.img_size, color_mode="grayscale")
#             y[j] = np.expand_dims(img, 2)
#             # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
#             y[j] -= 1
#         return x, y
    
    
#     def _load_single_data(self, patient):
#         ppg = np.load(f"{self.ppg_path}/mesa-sleep-{patient}-Pleth.npy")
#         ppg = sosfiltfilt(self.sos, ppg, axis=0)
#         F = self.fs / self.resample_fs
#         ppg = scipy.signal.resample(ppg, int(len(ppg) / F)).astype(np.float16)
#         return ppg
    
#     def _load_single_label(self, patient):
#         peaks = np.round(np.load(f"{self.peak_path}/{patient}.npy")/self.F).astype(int)
#         return peaks
    
#     def _load_data(self):
#         data = Parallel(n_jobs=64, prefer="threads")(delayed(self._load_single_data)(patient) for patient in tqdm.tqdm(self.patients))
#         return data
        
#     def _load_labels(self):
#         labels = [self._load_single_label(patient) for patient in tqdm.tqdm(self.patients)]
#         return labels
    
#     def _make_dataset_from_signal_and_labels(self):
#         window = self.resample_fs*self.window
#         hop = self.resample_fs*self.hop
#         shape_estimate = int(len(self.patients)*(11*60*60*32)/hop)
#         print(shape_estimate)
#         data = np.empty((shape_estimate,window), np.float16)
#         labels = np.empty((shape_estimate,window), int)
        
#         shape = 0
#         for i,patient in tqdm.tqdm(enumerate(self.patients)):
#             patient_data = self.data[i]
#             patient_labels = np.zeros_like(patient_data)
#             patient_labels[self.labels[i][:-2]]=1
            
#             strided_data = self._time_series_subsequences(patient_data, window, hop)
#             strided_labels = self._time_series_subsequences(patient_labels, window, hop)
            
#             data[shape:shape+strided_data.shape[0],:] = strided_data
#             labels[shape:shape+strided_data.shape[0],:] = strided_labels
#             shape = shape + strided_data.shape[0]
#             if shape>shape_estimate:
#                 print("Exceeded size")
#                 break
        
#         print(shape_estimate, shape, shape/shape_estimate)
#         return data[0:shape], labels[0:shape]
    

#     def _time_series_subsequences(self,ts, window, hop=1):
#         assert len(ts.shape) == 1
#         shape = (int(int(ts.size - window) / hop + 1), window)
#         strides = ts.strides[0] * hop, ts.strides[0]
#         return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)


In [5]:
# ppg_path = "/home/kkotzen/databases/mesa/polysomnography/signals/npy/Pleth"
# peak_path = "/home/kkotzen/leas_bishop_peaks/peaks"

# train_patients = [p.split(".")[0] for p in os.listdir(peak_path)][0:1000]
# validate_patients = [p.split(".")[0] for p in os.listdir(peak_path)][1001:1200]
# test_patients = [p.split(".")[0] for p in os.listdir(peak_path)][1201:1300]

# train_dataloader = PPGPeakOnsetLoader(32, 16, 6, ppg_path, peak_path, train_patients)
# d_train, l_train = train_dataloader._make_dataset_from_signal_and_labels()

# validate_dataloader = PPGPeakOnsetLoader(32, 16, 6, ppg_path, peak_path, validate_patients)
# d_validate, l_validate = validate_dataloader._make_dataset_from_signal_and_labels()

# test_dataloader = PPGPeakOnsetLoader(32, 16, 6, ppg_path, peak_path, test_patients)
# d_test, l_test = test_dataloader._make_dataset_from_signal_and_labels()

In [6]:
# np.save("/tmp/deep_peaks_signals_train.npy", d_train)
# np.save("/tmp/deep_peaks_labels_train.npy", l_train.astype(np.int8))

# np.save("/tmp/deep_peaks_signals_validate.npy", d_validate)
# np.save("/tmp/deep_peaks_labels_validate.npy", l_validate.astype(np.int8))

# np.save("/tmp/deep_peaks_signals_test.npy", d_test)
# np.save("/tmp/deep_peaks_labels_test.npy", l_test.astype(np.int8))

In [7]:
import numpy as np

d_train = np.load("/tmp/deep_peaks_signals_train.npy")
l_train = np.load("/tmp/deep_peaks_labels_train.npy")

d_validate = np.load("/tmp/deep_peaks_signals_validate.npy")
l_validate = np.load("/tmp/deep_peaks_labels_validate.npy")

d_test= np.load("/tmp/deep_peaks_signals_test.npy")
l_test = np.load("/tmp/deep_peaks_labels_test.npy")

In [8]:
# plt.close('all')
# plt.plot(d_train[500])
# plt.plot(np.where(l_train[500]==1)[0], d_train[500][np.where(l_train[500]==1)[0]], 'x')

In [9]:
#Before ever importing tensorflow 
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set the GPU you wish to use here
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], enable=True) #set the GPU you want to use here
print(gpus)

from tensorflow.keras.layers import Dense, Dropout, Reshape, Conv1D, BatchNormalization, Activation, AveragePooling1D, GlobalAveragePooling1D, Lambda, Input, Concatenate, Add, UpSampling1D, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.initializers import random_normal
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import Callback

from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import KFold, train_test_split

def cbr(x, out_layer, kernel, stride, dilation):
    x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def se_block(x_in, layer_n):
    x = GlobalAveragePooling1D()(x_in)
    x = Dense(layer_n//8, activation="relu")(x)
    x = Dense(layer_n, activation="sigmoid")(x)
    x_out=Multiply()([x_in, x])
    return x_out

def resblock(x_in, layer_n, kernel, dilation, use_se=True):
    x = cbr(x_in, layer_n, kernel, 1, dilation)
    x = cbr(x, layer_n, kernel, 1, dilation)
    if use_se:
        x = se_block(x, layer_n)
    x = Add()([x_in, x])
    return x  

def Unet(input_shape=(512,1)):
    layer_n = 32
    kernel_size = 7
    depth = 2

    input_layer = Input(input_shape)
    print('Input', input_layer.shape)
    input_layer_1 = AveragePooling1D(4)(input_layer)
    input_layer_2 = AveragePooling1D(16)(input_layer)
    
    ########## Encoder
    x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size, 1)
    out_0 = x

    x = cbr(x, layer_n*2, kernel_size, 4, 1)
    for i in range(depth):
        x = resblock(x, layer_n*2, kernel_size, 1)
    out_1 = x
    x = Concatenate()([x, input_layer_1])    
    x = cbr(x, layer_n*3, kernel_size, 4, 1)
    for i in range(depth):
        x = resblock(x, layer_n*3, kernel_size, 1)
    out_2 = x
    x = Concatenate()([x, input_layer_2])    
    x = cbr(x, layer_n*4, kernel_size, 4, 1)
    for i in range(depth):
        x = resblock(x, layer_n*4, kernel_size, 1)
    ########### Decoder
    x = UpSampling1D(4)(x)
    x = Concatenate()([x, out_2])
    x = cbr(x, layer_n*3, kernel_size, 1, 1)

    x = UpSampling1D(4)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n*2, kernel_size, 1, 1)

    x = UpSampling1D(4)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size, 1, 1)    

    #regressor
    #x = Conv1D(1, kernel_size=kernel_size, strides=1, padding="same")(x)
    #out = Activation("sigmoid")(x)
    #out = Lambda(lambda x: 12*x)(out)
    
    #classifier
    x = Conv1D(1, kernel_size=kernel_size, strides=1, padding="same")(x)
    out = Activation("sigmoid")(x)
    
    model = Model(input_layer, out)
    
    return model

2021-08-04 01:33:54.063057: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-04 01:33:57.878018: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-04 01:33:59.069226: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2021-08-04 01:33:59.069295: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-04 01:33:59.087759: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-04 01:33:59.087821: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
    
    idx_1 = np.arange(len(input_dataset))
    while True:
        np.random.shuffle(idx_1)
        for i in range(int(len(input_dataset)/batch_size)):
            inputs = input_dataset[idx_1[i*batch_size:(i+1)*batch_size]]
            targets = target_dataset[idx_1[i*batch_size:(i+1)*batch_size]]
            yield inputs, targets

class macroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = targets

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        f1_val = f1_score(self.targets, pred, average="macro")
        print("val_f1_macro_score: ", f1_val)
                
def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
    print(f"{train_inputs.shape=}")
    hist = model.fit_generator(
        Datagen(train_inputs, train_targets, batch_size, is_train=True),
        steps_per_epoch = len(train_inputs) // batch_size,
        epochs = n_epoch,
        validation_data=Datagen(val_inputs, val_targets, batch_size),
        validation_steps = len(val_inputs) // batch_size,
        callbacks = [lr_schedule, macroF1(model, val_inputs, val_targets)],
        shuffle = False,
        verbose = 1
        )
    return hist


def lrs(epoch):
    if epoch<35:
        lr = learning_rate
    elif epoch<50:
        lr = learning_rate/10
    else:
        lr = learning_rate/100
    return lr

In [11]:
K.clear_session()
model = Unet()
# print(model.summary())
# d_train
# l_train
# d_validate
# l_validate

# learning_rate=0.0005
# n_epoch=60
# batch_size=32

# lr_schedule = LearningRateScheduler(lrs)

# #regressor
# #model.compile(loss="mean_squared_error", 
# #              optimizer=Adam(lr=learning_rate),
# #              metrics=["mean_absolute_error"])

#classifier
learning_rate = 0.0001
model.compile(loss=categorical_crossentropy, 
              optimizer=Adam(lr=learning_rate), 
              metrics=["accuracy"])
model.fit(d_train, l_train, batch_size = 2048, epochs=5)

# hist = model_fit(model, d_train, l_train, d_validate, l_validate, n_epoch, batch_size)

Input (None, 512, 1)


2021-08-04 01:34:00.021739: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-04 01:34:00.034040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2021-08-04 01:34:00.035980: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2021-08-04 01:34:00.036035: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-04 01:34:00.852964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnec

Epoch 1/5


2021-08-04 01:34:41.990064: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-04 01:34:43.065583: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8200
2021-08-04 01:34:45.324834: W tensorflow/stream_executor/gpu/asm_compiler.cc:64] Running ptxas --version returned 32512
2021-08-04 01:34:46.685722: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-08-04 01:34:46.730551: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-04 01:34:47.826156: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-08-04 01:34:47.951573: I tensorflow/stream_executor/cuda/cu

3120/3120 [==============================] - 458s 142ms/step - loss: 132.3460 - accuracy: 0.9411
Epoch 2/5
3120/3120 [==============================] - 441s 141ms/step - loss: 128.6318 - accuracy: 0.9578
Epoch 3/5
3120/3120 [==============================] - 441s 141ms/step - loss: 128.4012 - accuracy: 0.9583
Epoch 4/5
2963/3120 [===========================>..] - ETA: 22s - loss: 128.2353 - accuracy: 0.9585

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



3120/3120 [==============================] - 441s 141ms/step - loss: 128.0879 - accuracy: 0.9587


In [15]:
preds = model.predict(d_validate, batch_size=512)


In [30]:
print(preds.shape)
plt.close('all')
fig, ax = plt.subplots(2,1)
ax[0].plot(l_validate[8000])
ax[0].plot(preds[8000], alpha=0.5)
ax[1].plot(d_validate[8000])

(1257446, 512, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …